# Save a scan as an .xy or .csv file

This notebook will save the azimuthally integrated data from a scan into either:
* a `.csv` file (each individual frame is saved as a column
* a `.xy` file with the mean of all frames (if the `averageData` flag is set)

It is also possible to save the meta data to a corresponding `_meta.csv`file (if the `saveMetaData` flag is set)

<div class="alert alert-block alert-danger">
<b>It is preffered (and more efficient) to work with the .h5 files if possible!</b><br>
Exporting the files to .csv more than doubles the filezisize.
<br>
<br>
<b>Do not export single frames from time scans to the MAX IV file system!</b>
</div>


In [ ]:
import DanMAX as DM
import numpy as np
import h5py
import os

In [ ]:
fname = DM.findScan(XXX)
saveMetaData = True
averageData = True

aname = DM.getAzintFname(fname)

dst = fname.replace('raw', 'process/csv').replace('.h5', '.csv')
if not os.path.exists(os.path.dirname(dst)):
    print(f'Creating folder: {os.path.dirname(dst)}')
    os.makedirs(os.path.dirname(dst))
    
with h5py.File(aname,'r') as f:
    try:
        x = f['q'][:]
        Q = True
    except KeyError:
        x = f['2th'][:]
        Q = False
    I = f['I'][:]

if Q:
    header = 'Q/AA^-1'
else:
    header = '2theta,' + ','.join([str(i+1) for i in range(I.shape[0])])
#Stack 2th/Q vector on intensity data and transpose - and write to file
if averageData:
    np.savetxt(dst.replace('.csv','_mean.xy'), np.vstack((x, np.mean(I, axis=0))).T)
else:
    np.savetxt(dst, np.vstack((x, I)).T, delimiter=',', fmt= '%.5f',header=header)

if saveMetaData:
    meta_dic = DM.getMetaDic(fname)
    keys = list(meta_dic.keys())
    header = ' '.join([f'{key:^24s}' for key in keys])
    data = np.array([meta_dic[key] for key in keys]).T
    np.savetxt(dst.replace('.csv','_meta.csv'), data, header=header,delimiter=',')